# 1-Shot Classifier

Classify images directly. Deep resnet.

In [ ]:
%matplotlib inline

# system libraries
import os
from glob import glob
import logging

# numerical,image and plotting stuff
import math
import pandas as pd
import numpy as np
from PIL import Image
from skimage import io
import skimage.transform as tf
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid", {'axes.grid' : False})

In [ ]:
# get all image file names and associated class from their latest location
class_folders = glob('data/train/*')
files = [glob(cls + '/*') for cls in class_folders] # put class info with file name
files = [img for cls in files for img in cls]
df = pd.DataFrame({'fpath':files})
df['category'] = df.fpath.str.extract('data/train/([a-zA-Z]*)/img', expand=False) # extract class
df.head()

In [ ]:
def read_and_augment(fpath,rng,w_out=512,h_out=512):
    # read image
    img = io.imread(fpath)
    h, w, _ = img.shape
    # prepare transformations
    r_tx, r_ty = -w/2, -h/2
    r_rotate = rng.uniform(-np.pi/6,np.pi/6,1)[0]
    r_scale = rng.uniform(0.9,1.1,1)
    tf_rotate = tf.SimilarityTransform(rotation=r_rotate)
    tf_scale = tf.SimilarityTransform(scale=r_scale)
    tf_shear = tf.AffineTransform(shear=rng.uniform(-0.1,0.1,1))
    tf_shift = tf.SimilarityTransform(translation=[-r_tx, -r_ty])
    tf_shift_inv = tf.SimilarityTransform(translation=[r_tx, r_ty])
    trans = tf_shift + tf_scale + tf_shear + tf_rotate + tf_shift_inv

    img_warped = tf.warp(img, trans.inverse)
    img_resized = tf.resize(img_warped,(h_out,w_out))/128.-1
    # randomly flip horizontally and vertically
    if rng.uniform(0,1) < 0.5:
        img_resized = img_resized[::-1,:,:]
    if rng.uniform(0,1) < 0.5:
        img_resized = img_resized[:,::-1,:]
    return img_resized


In [ ]:
def gen(df,batch_size=64,rng=None,h_out=512,w_out=512,ch_out=3):
    """
    This generator produces a batch of (X,y) for training a fish detector.
    Images are randomly augmented.
    
    Inputs:
    df is a pandas dataframe containing the file path (fpath) and class
    
    Outputs:
    X is a 4D tensor of shape (batch_size,h,w,ch)
    y is a 2D vector of (batch_size,8), for the 8 classes
    """

    if rng is None:
        rng = np.random.RandomState()

    n = len(df)
    total_batch = int(np.ceil(n / batch_size))
    logging.info('generating %d batches with %d samples per epoch' % (total_batch,n))
    
    # initialize labelBinarizer
    ohc = LabelBinarizer()
    ohc.fit(df.category)

    while True:
        # shuffle examples every epoch
        df_shuffled = df.iloc[rng.permutation(n)]
        for i_batch in range(total_batch):
            # limit end index by size of df_gen to prevent 
            # indexing up to the next multiple of batch_size
            i_start, i_end = i_batch * batch_size, min((i_batch + 1) * batch_size,n)
            i_batch_size = i_end - i_start
            X = np.zeros((i_batch_size,h_out,w_out,ch_out))
            Y = np.zeros((i_batch_size,8))
            for i in range(i_start,i_end):
                fpath = df_shuffled.iloc[i]['fpath']
                x = read_and_augment(fpath,rng,w_out,h_out) # read and augment image
                i_intrabatch = i - i_start
                X[i_intrabatch,...] = x
            Y = ohc.transform(df_shuffled.iloc[i_start:i_end]['category'])
            logging.info('yielding batch %d of size %d' % (i_batch, i_batch_size))
            yield(X,Y)

In [ ]:
# initialize generators
rng = np.random.RandomState(290615)

# we just use train valid test for training our model
ix_coord = rng.choice(range(3),p=[0.8,0.1,0.1],size=len(df))
df_train = df.ix[ix_coord==0]
df_valid = df.ix[ix_coord==1]
df_test = df.ix[ix_coord==2]
gn_train = gen(df_train,batch_size=64,rng=np.random.RandomState(290615),h_out=512,w_out=512)
gn_valid = gen(df_valid,batch_size=64,rng=np.random.RandomState(290615),h_out=512,w_out=512)
samples_per_epoch = len(df_train)
nb_val_samples = len(df_valid)


In [ ]:
from keras.layers import Input, Convolution2D, MaxPooling2D, ZeroPadding2D, Layer,\
    Activation, Dropout, Flatten, AveragePooling2D, Dense, merge
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.optimizers import Adam
from keras import backend as K
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau

# setup model
h_out, w_out = 512, 512

# creates a residual block
def res_block(input_layer,depth,layer_number,if_pool=True):
    # skip bn-relu for first layer (0-indexed here)
    if layer_number > 0:
        bn = BatchNormalization()(input_layer)
        bn_relu = Activation('relu')(bn)
    else:
        bn_relu = input_layer
    bn_relu_conv = Convolution2D(depth,3,3,name='conv'+str(layer_number)+'_1',
                               border_mode='same',W_regularizer=l2(0.0001))(bn_relu)
    bn_relu_conv_bn = BatchNormalization()(bn_relu_conv)
    bn_relu_conv_bn_relu = Activation('relu')(bn_relu_conv_bn)
    bn_relu_conv_bn_relu_conv = Convolution2D(depth,3,3,name='conv'+str(layer_number)+'_2',
                               border_mode='same',W_regularizer=l2(0.0001))(bn_relu_conv_bn_relu)
    residual = Convolution2D(depth,1,1,name='resid'+str(layer_number),border_mode='same',
                             W_regularizer=l2(0.0001))(input_layer)
    merged = merge([bn_relu_conv_bn_relu_conv,residual],mode='sum')
    merged_pool = MaxPooling2D((2,2),strides=(2,2))(merged)
    return merged_pool


def new_model(h_in=512, w_in=512 ,ch=3):
    
    image_input = Input(shape=(512,512,3))

    res1_out = res_block(image_input,16,0,True)
    res2_out = res_block(res1_out,32,1,True)
    res3_out = res_block(res2_out,64,2,True)
    res4_out = res_block(res3_out,128,3,True)
    res5_out = res_block(res4_out,128,4,False)
    res6_out = res_block(res5_out,256,5,True)
    res7_out = res_block(res6_out,512,6,False)
    res7_avg = AveragePooling2D(pool_size=(4,4))(res7_out)
    post_conv_flat = Flatten()(res5_out)

    post_conv_flat = Dropout(0.5)(post_conv_flat)
    post_conv_flat = Dense(1024, activation='relu', init='glorot_normal')(post_conv_flat)
    post_conv_flat = Dropout(0.5)(post_conv_flat)
    predictions = Dense(8, activation='softmax', init='glorot_normal')(post_conv_flat)
    
    model = Model(input=[image_input],output=[predictions])
    return model

model = new_model()
adam = Adam(lr=0.001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])


# setup tensorboard graph directory
graph_dir = 'graph/graph_1shot'
if not os.path.exists(graph_dir):
    os.makedirs(graph_dir)

# prepare callbacks
tb = TensorBoard(log_dir=graph_dir, write_graph=True, write_images=True)
mc_coord = ModelCheckpoint(filepath='models/classifier_ep{epoch:02d}_loss{val_loss:.2f}_acc{val_acc:.2f}.h5',
                           verbose=1,save_best_only=True)
reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=5, 
                             verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=1e-7)

In [ ]:
model.fit_generator(generator=gn_train,
                        samples_per_epoch=samples_per_epoch,
                        validation_data=gn_valid,
                        nb_val_samples=nb_val_samples,
                        nb_epoch=500,
                        callbacks=[tb,mc_coord,reducelr])

# Prepare Submission

In [ ]:
# read in data files
files = glob('data/test_stg2/*')
df = pd.DataFrame({'fpath':files,'hx':np.nan,'hy':np.nan,'tx':np.nan,'ty':np.nan})
print(len(df))
df.head()

model.load_weights('models/classifier_ep126_loss0.17_acc0.95.h5')